In [ ]:
### Set up the root folder
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/GPT

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/GPT


In [ ]:
!pip install -r ./requirements.txt -qq
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets


In [ ]:
import transformers
import pandas as pd
from model import GPT2PPLV2 as GPT2PPL
from sklearn.metrics import accuracy_score
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
datasetname = [ 'DagPap22','chatgpt_human_combined','IELTS_GPT35_Human_Comparison']

detectgptmodelname = 'detectgpt'
gptzeromodelname = 'gptzero'

def getInputfileName(datasetname):
    return datasetname+'_Corpus.csv'

def getOutputfileName(datasetname, modelname):
    return datasetname+'_result_'+modelname+'.csv'

In [1]:
#function to process the detectGPT and Chat GPT model on a given datasets file
def processGPTModel(datasetFile):
    df_combined = pd.read_csv(datasetFile)
    # initialize the model
    model = GPT2PPL()

    sentence = """Some people believe that a college or a university should be available to all students. Others think that higher education should be available only to good students. In my opinion, higher education should be available to all students. Why do I think so?
    Firstly, to get higher education is a right of any person. If higher education is available only to good student, it will be infringement of human rights. We cannot say to man that she or he is not entitled to get higher education because it will be anti-constitutional. On the other hand, a college or university can have own rules for students who don't have a good GPA. For example, students can graduate from college if they have only "A" or "B". Another example, students can take several times the same exam while they won’t receive a good grade. It is very important to give a right to get higher education to all students, because it will be their chance to change something in their life, get better job in the future or built their career.
    Secondly, who gives us right to share people in good and bad students. Maybe they have good abilities for studying, yet, these students chose the wrong way ignoring learning when they were high school students. We have a lot of examples when people didn't have a good grade when they were a high school students, however, they could graduate from college and became to work successfully. We know many examples about great people, Pushkin, Mendeleev, who weren't good students in all subjects, but they were very talented persons and whole world knows their names. To be a good or bad students are not the same to have abilities for learning. We should understand and remember it.
    Some people can say that government and parents just waste money for education bad students. Nonetheless, the government also can change rules for those students. For example, not support financially if the student has a grade "C" or worse. However, our government sustains any students. I think that it is the right way, because we live in a free country and any man has the same right to get higher education. In summary, in spite of colleges and universities are mostly interested in good students, I strongly believe that any students, good or bad, should have the same right to earn high education."""

    temp = model(sentence, 200, "v1.1")
    
    df_combined['token_count']= [len(tokenize.word_tokenize(x)) for x in df_combined.text]
    result_detectgpt = pd.DataFrame(columns=['sr_no','prob', 'label', 'desc'])
    result_gptzero = pd.DataFrame(columns=['sr_no','Perplexity', 'Perplexity per line', 'Burstiness', 'label', 'desc'])

    #“perplexity” (aka randomness) and “burstiness” (aka variance)
    # GPTZero:
    # To determine whether an excerpt is written by a bot, GPTZero uses two indicators: "perplexity" and "burstiness."

    # Perplexity
    # Perplexity measures the complexity of text; if GPTZero is perplexed by the text, then it has a high complexity and it's more likely to be human-written.
    # However, if the text is more familiar to the bot — because it's been trained on such data — then it will have low complexity and therefore is more likely to be AI-generated.

    # “burstiness”
    # If a term is used once in a document, then it is likely to be used again. This phenomenon is called burstiness, and it implies that the second and later appearances of a word are less significant than the first appearance

    for i in range(len(df_combined)):
      print(i)
      try:
        token_count = min(df_combined['token_count'][i], 200)
        pred, desc = model(df_combined.text[i], token_count, "v1.1")
        pred['desc']= desc
        pred['sr_no']= df_combined.sr_no[i]
      except:
        pred['prob']= None
        pred['label']= None
        pred['desc']= 'error'
        pred['sr_no']= df_combined.sr_no[i]
      result_detectgpt = result_detectgpt.append(pred, ignore_index=True)

      try:
        pred, desc = model(df_combined.text[i], token_count, "v1")
        pred = dict(pred)
        pred['desc']= desc
        pred['sr_no']= df_combined.sr_no[i]
      except:
        pred['Perplexity']= None
        pred['Perplexity per line']= None
        pred['Burstiness']= None
        pred['label']= None
        pred['desc']= 'error'
        pred['sr_no']= df_combined.sr_no[i]
      result_gptzero = result_gptzero.append(pred, ignore_index=True)
      result_detectgpt.to_csv(getOutputfileName(datasetFile, detectgptmodelname))
      result_gptzero.to_csv(getOutputfileName(datasetFile, gptzeromodelname))
      
      result_detectgpt['label']=result_gptzero.label.map({1:'human', 0:'chatgpt'})
      result_gptzero['label']=result_gptzero.label.map({1:'human', 0:'chatgpt'})

      gptzero_final=pd.concat([result_gptzero, df_combined], axis=1)
      gptzero_final = gptzero_final[['sr_no', 'source', 'label']].dropna()
      detectgpt_final=pd.concat([result_gptzero, df_combined], axis=1)
      detectgpt_final = detectgpt_final[['sr_no', 'source', 'label']].dropna()

      from sklearn.metrics import accuracy_score, classification_report### DetectGPT
      print("### DetectGPT")
      print(classification_report(detectgpt_final['source'], detectgpt_final['label']))
      print("### GPTZero")
      print(classification_report(gptzero_final['source'], gptzero_final['label']))

In [ ]:
#Process for  all datasets one by one
for datasetfile in datasetname:
    processGPTModel(datasetfile)
    

##  code lines below can be deleted

In [ ]:
df_combined = pd.read_csv("DagPap22_Corpus.csv")

In [ ]:
display(df_combined)

Unnamed: 0  sr_no                                               text  \
0             0  16811  A scalable elliptic curve cryptography (ECC) p...   
1             1  16513  Design of dual port memory in QCA is an intere...   
2             2  26583  The rapid increase in the use of microprocesso...   
3             3    635  Event counting is a common function performed ...   
4             4  19911  In delivering multimedia services, quality of ...   
..          ...    ...                                                ...   
227         227  25613  In this paper we extend the notion of minimum ...   
228         228  26106  In this paper, we develop a linear programming...   
229         229  26376  The effect of parental care in a social specie...   
230         230  26584  In this paper, we supplement the work of Hall ...   
231         231  26689  Recent studies have shown that the genetic var...   

      source  fake       source_org  
0      human     0  micpro_original  
1      human     0  micpro_original  
2      human     0  micpro_original  
3      human     0  micpro_original  
4      human     0  micpro_original  
..       ...   ...              ...  
227  chatgpt     1   generated_gpt3  
228  chatgpt     1   generated_gpt3  
229  chatgpt     1   generated_gpt3  
230  chatgpt     1   generated_gpt3  
231  chatgpt     1   generated_gpt3  

[232 rows x 6 columns]

In [ ]:
# from detectgpt_gptzero import similar, normCdf, likelihoodRatio, GPT2PPLV2

In [ ]:
# initialize the model
model = GPT2PPL()

sentence = """Some people believe that a college or a university should be available to all students. Others think that higher education should be available only to good students. In my opinion, higher education should be available to all students. Why do I think so?
Firstly, to get higher education is a right of any person. If higher education is available only to good student, it will be infringement of human rights. We cannot say to man that she or he is not entitled to get higher education because it will be anti-constitutional. On the other hand, a college or university can have own rules for students who don't have a good GPA. For example, students can graduate from college if they have only "A" or "B". Another example, students can take several times the same exam while they won’t receive a good grade. It is very important to give a right to get higher education to all students, because it will be their chance to change something in their life, get better job in the future or built their career.
Secondly, who gives us right to share people in good and bad students. Maybe they have good abilities for studying, yet, these students chose the wrong way ignoring learning when they were high school students. We have a lot of examples when people didn't have a good grade when they were a high school students, however, they could graduate from college and became to work successfully. We know many examples about great people, Pushkin, Mendeleev, who weren't good students in all subjects, but they were very talented persons and whole world knows their names. To be a good or bad students are not the same to have abilities for learning. We should understand and remember it.
Some people can say that government and parents just waste money for education bad students. Nonetheless, the government also can change rules for those students. For example, not support financially if the student has a grade "C" or worse. However, our government sustains any students. I think that it is the right way, because we live in a free country and any man has the same right to get higher education. In summary, in spite of colleges and universities are mostly interested in good students, I strongly believe that any students, good or bad, should have the same right to earn high education."""

temp = model(sentence, 200, "v1.1")

In [ ]:
df_combined['token_count']= [len(tokenize.word_tokenize(x)) for x in df_combined.text]

In [ ]:
result_detectgpt = pd.DataFrame(columns=['sr_no','prob', 'label', 'desc'])
result_gptzero = pd.DataFrame(columns=['sr_no','Perplexity', 'Perplexity per line', 'Burstiness', 'label', 'desc'])

#“perplexity” (aka randomness) and “burstiness” (aka variance)
# GPTZero:
# To determine whether an excerpt is written by a bot, GPTZero uses two indicators: "perplexity" and "burstiness."

# Perplexity
# Perplexity measures the complexity of text; if GPTZero is perplexed by the text, then it has a high complexity and it's more likely to be human-written.
# However, if the text is more familiar to the bot — because it's been trained on such data — then it will have low complexity and therefore is more likely to be AI-generated.

# “burstiness”
# If a term is used once in a document, then it is likely to be used again. This phenomenon is called burstiness, and it implies that the second and later appearances of a word are less significant than the first appearance

for i in range(len(df_combined)):
  print(i)
  try:
    token_count = min(df_combined['token_count'][i], 200)
    pred, desc = model(df_combined.text[i], token_count, "v1.1")
    pred['desc']= desc
    pred['sr_no']= df_combined.sr_no[i]
  except:
    pred['prob']= None
    pred['label']= None
    pred['desc']= 'error'
    pred['sr_no']= df_combined.sr_no[i]
  result_detectgpt = result_detectgpt.append(pred, ignore_index=True)

  try:
    pred, desc = model(df_combined.text[i], token_count, "v1")
    pred = dict(pred)
    pred['desc']= desc
    pred['sr_no']= df_combined.sr_no[i]
  except:
    pred['Perplexity']= None
    pred['Perplexity per line']= None
    pred['Burstiness']= None
    pred['label']= None
    pred['desc']= 'error'
    pred['sr_no']= df_combined.sr_no[i]
  result_gptzero = result_gptzero.append(pred, ignore_index=True)

In [ ]:
result_detectgpt.to_csv("DAGPap22_result_detectgpt.csv")
result_gptzero.to_csv("DAGPap22_result_gptzero.csv")

In [ ]:
result_detectgpt['label']=result_gptzero.label.map({1:'human', 0:'chatgpt'})
result_gptzero['label']=result_gptzero.label.map({1:'human', 0:'chatgpt'})

In [ ]:
gptzero_final=pd.concat([result_gptzero, df_combined], axis=1)
gptzero_final = gptzero_final[['sr_no', 'source', 'label']].dropna()
detectgpt_final=pd.concat([result_gptzero, df_combined], axis=1)
detectgpt_final = detectgpt_final[['sr_no', 'source', 'label']].dropna()

In [ ]:
from sklearn.metrics import accuracy_score, classification_report### DetectGPT
print("### DetectGPT")
print(classification_report(detectgpt_final['source'], detectgpt_final['label']))
print("### GPTZero")
print(classification_report(gptzero_final['source'], gptzero_final['label']))